<a href="https://colab.research.google.com/github/mazizf13/Prediksi-Tingkat-Pengangguran-Terbuka-TPT-di-Indonesia/blob/main/Prediksi_Tingkat_Pengangguran_Terbuka_(TPT)_di_Indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prediksi Tingkat Pengangguran Terbuka (TPT) di Indonesia**

## **Pendefinisian fungsi dan kelas yang digunakan**

In [1]:
# Import library keras dan lain-lain

import numpy as np # For matrix math
from numpy import loadtxt
from keras.models import Sequential
import matplotlib.pyplot as plt # For plotting
import keras
from keras.utils import to_categorical
from keras.layers import Flatten, Dense, Activation
from IPython.display import clear_output

## **Normalisasi Data**

In [ ]:
def normalisasi(x):
    a = np.max(x)
    b = np.min(x)
    hasil_normalisasi = (0.8 * (x - b)) / (a - b) + 0.1
    return hasil_normalisasi

In [ ]:
# load the data latih
# Data Tingkat Pengangguran Terbuka (TPT) mulai Tahun 1997
# sampai dengan Tahun 2020

TPT = loadtxt('data_pengangguran.csv', delimiter=',')
TPT_normal = normalisasi(TPT)

TPT_Target = TPT[:,10]
TPT_Target_Uji = TPT[:,11]

# DATA PELATIHAN =================================
# Data dibagi menjadi 10 variabel input (X) dan 1 variabel output (Y)
# Input : mulai tahun 1997 sampai tahun 2016

X = TPT_normal[:,0:10]

# Output : data TPT tahun 2017 dan 2018

Y = TPT_normal[:,10]

# DATA UJI PREDIKSI=================================
# Geser dua tahun ke depan
# Input : mulai tahun 1999 sampai tahun 2018
# Untuk memprediksi TPT Tahun 2019 dan 2020

X1 = TPT_normal[:,1:11]

# Urut tahun, menggantikan indeks dari 0 s.d 1

urut = [1,
 2
 ]

In [ ]:
print(TPT)

In [ ]:
TPT_normal = normalisasi(TPT)
print(TPT_normal)

In [ ]:
baris, kolom = TPT.shape
print("barisnya =",baris)
print("kolomnya =",kolom)

In [ ]:
baris, kolom = TPT_normal.shape
print("barisnya =",baris)
print("kolomnya =",kolom)

### **Masukkan Jaringan X**

In [ ]:
baris, kolom = X.shape
print("barisnya =",baris)
print("kolomnya =",kolom)

In [ ]:
# Mendefinisikan model keras

model = Sequential()
model.add(Dense(10, input_dim=10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Proses kompilasi (pelatihan) Deep Learning

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [ ]:
class PlotLosses(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.i = 0
    self.x = []
    self.losses = []
    self.val_losses = []

    self.fig = plt.figure()

    self.logs = []

  def on_epoch_end(self, epoch, logs={}):

    self.logs.append(logs)
    self.x.append(self.i)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))
    self.i += 1

    clear_output(wait=True)
    plt.plot(self.x, self.losses, label="loss")
    plt.plot(self.x, self.val_losses, label="val_loss")
    plt.legend()
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show();

plot_losses = PlotLosses()

In [ ]:
model.fit(X, Y,
 epochs=500,
 validation_data=(X, Y),
 callbacks=[plot_losses],
 verbose=0)

In [ ]:
# evaluate the keras model
# Evalusi hasil proses pelatihan Deep Learning
# Hitung akurasi hasil pelatihan dalam bentuk Prosentase Keberhasilan
# Prosentase Keberhasilan = 100 - MAPE

#_, MAPE_pelatihan = model.evaluate(X_normal, Y_normal, batch_size=1)

_, MSE = model.evaluate(X, Y, batch_size=1)

#print('Prosentase_keberhasilan: %.2f' % (100-MAPE_pelatihan))

print('MSE : %.2f' % (MSE))

In [ ]:
# menjalankan fungsi prediksi

y_pred = model.predict(X)
print(y_pred)

In [ ]:
# Denormalisasi Data sebelum dibandngkan

a = np.max(TPT)
b = np.min(TPT)

# Rumus
# x_denormalisasi = (((x_normalisasi - 0.1)*(a - b))/0.8) + b

y_prediksi = (((y_pred - 0.1)*(a - b))/0.8) + b
print("y_prediksi =", y_prediksi)

In [ ]:
# Urut tahun, menggantikan indeks dari 0 s.d 1

urutan1 = ['2017',
 '2018'
 ]

In [ ]:
# Plot Keluaran Hasil Prediksi Tahap Pelatihan

plt.title('Training Stage Prediction Results')
plt.plot(urutan1, TPT_Target, '-b', label='Target', linewidth=3, markersize=12)
plt.plot(urutan1, y_prediksi, '--r', label='Prediction', linewidth=3, markersize=12)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Open Unemployment Rate (OUR) (%)')
plt.ylim((2,8))
plt.grid()
plt.show()

In [ ]:
# Plot Kesalahan Prediksi

selisih = TPT_Target - y_prediksi.transpose()
plt.title('Training Stage Prediction Error')
plt.plot(urutan1, selisih.transpose(), '-m', label='error', linewidth=3, markersize=12)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Open Unemployment Rate (OUR) (%)')
plt.ylim((-2.5, 2.5))
plt.grid()
plt.show()

In [ ]:
# Pengukuran kinerja Deep Learning

from sklearn.metrics import mean_squared_error
from math import sqrt

# Menghitung Mean Squared Error (MSE)
mse = mean_squared_error(TPT_Target, y_prediksi)
rmse = sqrt(mse)
print("MSE =", mse)
print("RMSE =", rmse)

# Fungsi untuk menghitung Mean Absolute Percentage Error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Menghitung MAPE
MAPE = mean_absolute_percentage_error(TPT_Target, y_prediksi)
print("MAPE =", MAPE)

# Menghitung Persentase Keberhasilan
Persentase_keberhasilan = 100 - MAPE
print("Persentase Keberhasilan =", Persentase_keberhasilan)

# **Prediksi TPT**

## **Tahap Pengujian**

#### Input : mulai tahun 1999 sampai tahun 2018
#### Untuk Memprediksi TPT di Tahun 2019 dan 2020

In [ ]:
# Urut tahun, menggantikan indeks dari 0 s.d 1

urutan2 = ['2019',
 '2020'
 ]

In [ ]:
# Ujicoba prediksi

y_uji = model.predict(X1)
print('Hasil prediksi sebelum denormalisasi =', y_uji)

In [ ]:
# Denormalisasi Data sebelum dibandngkan

a = np.max(TPT)
b = np.min(TPT)

# Rumus
# x_denormalisasi = (((x_normalisasi - 0.1)*(a - b))/0.8) + b

y_pengujian = (((y_uji - 0.1)*(a - b))/0.8) + b

print("y_pengujian =", y_pengujian)

In [ ]:
# Plot keluaran hasil prediksi Tahap Pengujian

plt.title('Testing Stage Prediction Results')
plt.plot(urutan2, TPT_Target_Uji, '-b', label='Desired', linewidth=3, markersize=12)
plt.plot(urutan2, y_pengujian, '--r', label='Prediction', linewidth=3, markersize=12)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Open Unemployment Rate (OUR) (%)')
plt.ylim((0, 10))
plt.grid()
plt.show()

In [ ]:
# Plot kesalahan prediksi Tahap Pengujian

# Menghitung selisih (kesalahan) prediksi
selisih_pengujian = TPT_Target_Uji - y_pengujian.transpose()

# Membuat plot kesalahan prediksi pada tahap pengujian
plt.title('Testing Stage Prediction Error')
plt.plot(urutan2, selisih_pengujian.transpose(), '-m', label='error', linewidth=3, markersize=12)
plt.legend()
plt.xlabel('Year')
plt.ylabel('Open Unemployment Rate (OUR) (%)')
plt.ylim((-7, 7))
plt.grid()
plt.show()

In [ ]:
# Pengukuran kinerja Deep Learning Tahap Pengujian

from sklearn.metrics import mean_squared_error
from math import sqrt

mse1 = mean_squared_error(TPT_Target_Uji, y_pengujian)
rmse1 = sqrt(mse1)
print("mse =",mse1)
print("rmse =",rmse1)

MAPE = mean_absolute_percentage_error(TPT_Target_Uji, y_pengujian)
print("MAPE =",MAPE)
Persentase_keberhasilan = 100-MAPE
print("Persentase_keberhasilan =", Persentase_keberhasilan)